# Best Books Ever

#### Audiencia y motivacion

Como un acérrimo lector siempre me encuentro en la busqueda de nuevos titulos para leer y recomendando a amigos y conocidos sus posibles nuevas lecturas. 
En ambos casos es comun que se presente la misma problematica: es muy dificil poder dar con lo que busco, generalmente por falta de conocimiento sobre nuevos lanzamientos o libros no tan nuevos pero que fueron escritos por autores que aún no tuve la oportunidad de conocer. Por esto, apenas descubri el dataset con el que estoy trabajando se me ocurrio hacer un sistema de recomendacion de libros basado en las puntuaciones que un usuario le ponga a titulos que ya leyó.

---

#### Descripcion

Best Books Ever es un data set extraido de la pagina GoodReads en donde los usuarios (entre otras cosas) pueden calificar los libros que hayan leido, este dataset recopila todas las puntuaciones para cada libro ademas de agregar informacion de los mismos como el genero al que pertenecen, la fecha de publicacion o el formato en el que fue publicado el libro.
Las columnas con las que cuento son:


| Columna | Descripcion |
| -------------- | ------------- |
| bookId | ID del libro como en goodreads.com |
| title | Titulo |
| author | Autor/a |
| rating | Calificacion global en goodreads |
| language | Idioma |
| genres | Lista de generos | 
| bookFormat | Tipo de encuadernado |
| pages | Cantidad de paginas |
| publishDate | Fecha de publicacion |
| firstPublishDate | Fecha de publicacion de la primer edicion |
| numRatings | Cantidad de calificaciones |
| likedPercent | Porcentaje de calificaciones mayores a dos estrellas |
| price | Precio |
| publishDecade | Decada de publicacion |
| weightedRating | Rating aplicando shrinkage estimation |


---

#### Carga de datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('BestBooksDS.csv', index_col=0)

In [3]:
df.head(3)

,bookId,title,author,rating,language,genres,bookFormat,pages,publishDate,firstPublishDate,numRatings,likedPercent,price,publishDecade,weightedRating
0,2767052-the-hunger-games,The Hunger Games,Suzanne Collins,4.33,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",hardcover,374,2008-09-14,NaN,6376780,96.0,5.09,2000,4.329518
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",paperback,870,2004-09-28,2003-06-21,2507623,98.0,7.38,2000,4.498101
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,English,"['Classics', 'Fiction', 'Historical Fiction', ...",paperback,324,2006-05-23,2060-07-11,4501075,95.0,5.62,2000,4.279428


---

#### Validacion del modelo

Al tratarse de un sistema de recomendaciones que utiliza un modelo de procesamiento de lenguaje natural basado en content-based filtering (ya que solo se trata de ver las similaridades entre distintos libros, no de predecir resultados en base a ratings de distintos usuarios como suelen hacer este tipo de sistemas), no poseo un set de datos etiquetado ni una variable objetivo, esto hace imposible decir si una recomendacion es o no buena de forma automatica ya que el valor de estas es totalmente subjetivo. Luego de mucha investigación, la unica forma de validacion que encontre es verificar manualmente la similaridad entre dichos libros. 

Por esto, continuando con lo trabajado en la entrega anterior probe modificar la forma de calcular la similaridad entre los vectores de palabras y tambien hacer un poco de ajuste de  hiperparametros, y comparar los resultados obtenidos para ver si puedo mejorar la performance del modelo.

---

#### Feature Engineering
usando las variables author, genres y publishDecade

In [4]:
# creo una copia del dataframe original
metadata = df.copy()

In [5]:
from ast import literal_eval

features = ['author', 'genres']

metadata['genres'] = metadata['genres'].apply(literal_eval)

In [6]:
# funcion que elimina los espacios, transforma a minuscula y en el caso de strings que contengan una coma, elimina todo el texto a partir de esta.
def cleaner(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", "").split(",")[0])
        else:
            return ''

In [7]:
# limpio las columnas 'author' y 'genres' con la funcion que acabo de crear
for feature in features:
    metadata[f'{feature}Clean'] = metadata[feature].apply(cleaner)

In [8]:
# funcion que devuelve un string concatenando las columnas 'author', 'publishDecade' y 'genres' del dataset utilizado con un espacio.
def make_soup(x):
    return ''.join(x['authorClean']) + ' '  +  ''.join(str(x['publishDecade'])) + ' ' + ' '.join(x['genresClean'])

In [9]:
# guardo la informacion devuelta por la funcion 'make_soup' en una nueva columna 'soup' del dataset
metadata['soup'] = metadata.apply(make_soup, axis=1)

In [10]:
#asi se ven estos resultados
metadata['soup'].head()

0    suzannecollins 2000 youngadult fiction dystopi...
1    j.k.rowling 2000 fantasy youngadult fiction ma...
2    harperlee 2000 classics fiction historicalfict...
3    janeausten 2000 classics fiction romance histo...
4    stepheniemeyer 2000 youngadult fantasy romance...
Name: soup, dtype: object

In [11]:
metadata.drop(columns=['bookId', 'rating', 'language', 'genres', 'bookFormat', 'pages', 'publishDate', 'firstPublishDate', 'numRatings', 'likedPercent', 'price', 'authorClean', 'genresClean'], inplace=True)

In [12]:
for i in range(len(metadata)):
    metadata['author'].iloc[i] = metadata['author'].iloc[i].split('(')[0]

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_6360\3045558770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['author'].iloc[i] = metadata['author'].iloc[i].split('(')[0]


In [13]:
len(metadata)

52429

In [14]:
metadata.drop_duplicates(subset=['title'],inplace=True)

In [15]:
metadata.to_csv('metadata.csv')

---

#### Creacion del modelo de content-based filtering

y probando distintas formas de medir la similaridad entre los vectores de texto e hyper tunning de parametros

In [17]:
# creo un vector de relacion usando la columna recien creada
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [18]:
count_matrix.shape

(49927, 24645)

In [13]:
# uso la funcion cosine_similarity para calcular la similaridad entre los vectores de relacion
from sklearn.metrics.pairwise import cosine_similarity
import time

start_time = time.time()
cos_sim = cosine_similarity(count_matrix, count_matrix)
print(f'Tiempo de ejecucion: {time.time() - start_time} segundos')

Tiempo de ejecucion: 366.59633779525757 segundos


In [19]:
from sklearn.metrics.pairwise import polynomial_kernel

poly_sim = polynomial_kernel(count_matrix, count_matrix)

In [20]:
import joblib

joblib.dump(poly_sim, 'poly_sim_model.pkl')

['poly_sim_model.pkl']

In [14]:
# reinicio los indices del dataframe metadata y creo una serie con los titulos e id de cada libro para poder relacionar los resultados de las predicciones con dicho dataframe
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [12]:
# creo una funcion que recibe un titulo como input y devuelve los 10 libros mas similares
def get_recommendations(title, scores):
    #encuentro el indice del titulo ingresado
    idx = indices[title]

    #busco los scores de similaridad con este libro
    sim_scores = list(enumerate(scores[idx]))

    #ordeno la lista de libros segun el score obtenido por cada uno
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #devuelvo los 10 mas similares (salteo el 1ro ya que el libro ingresado es siempre el mas similar a si mismo)
    sim_scores = sim_scores[1:11]

    #busco los indices
    movie_indices = [i[0] for i in sim_scores]

    #devuelvo los titulos de los libros mas similares
    return metadata['title'].iloc[movie_indices]

---

#### Hypertunning de Parametros

Desepues de comparar los resultados de ambas formas de medir la similaridad de los vectores de texto se puede observar como los resultados son en su mayoria muy similares. la principal diferencia entre estas metricas es el tiempo de procesamiento, llevando el polynomial kernel un total de 679 segundos (11 minutos aprox.) y el cosine similarity un total de 370 segundos (6 minutos aprox.) llevandome esto a decidirme por usar cosine similarity.

Una vez seleccionada la metrica a usar voy a hacer hypertunning de parametros y comparar los resultados.

In [24]:
#genero 10 numeros aleatorios para usar como indices de libros que luego utilizare para evaluar los resultados
idx_test = np.random.random_integers(0, len(df), 4)

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_9336\4125564033.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 52429 + 1) instead
  idx_test = np.random.random_integers(0, len(df), 4)


In [25]:
libros_test = []

#busco los titulos de los libros que tengan un index de los generados anteriormente
for idx in idx_test:
    libros_test.append(df['title'].iloc[idx])

print(idx_test)
print(libros_test)

[38463 30246  4610 25625]
['The Tin Woodman of Oz', 'Το Αυτό-Τόμος ΙΙ', 'Hidden Figures', 'Hunting Annabelle']
